훈련 데이터가 부족한 상황이라면 모델에 파이토치의 nn.Embedding()을 사용하는 것보다 다른 텍스트 데이터로 사전 훈련되어 있는 임베딩 벡터를 불러오는 것이 나은 선택일 수 있습니다. 일반적이고 보다 많은 훈련 데이터로 이미 Word2Vec이나 GloVe 등으로 학습되어져 있는 임베딩 벡터들을 사용하는 것이 성능의 개선을 가져올 수 있습니다.

# IMDB 리뷰 데이터를 훈련 데이터로 사용하기

In [2]:
from torchtext import data, datasets

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다

In [ ]:
print('훈련 데이터의 크기 : {}' .format(len(trainset)))

In [ ]:
print(vars(trainset[0]))

# 토치텍스트를 사용한 사전 훈련된 워드 임베딩

## 사전 훈련된 Word2Vec 모델 확인하기

In [ ]:
from gensim.models import KeyedVectors

word2vec_model = KeyedVectors.load_word2vec_format('eng_w2v') # '영어/한국어 Word2Vec 훈련하기 챕터'에서 만들어두었던 'eng_w2v' 모델을 사용

In [ ]:
print(word2vec_model['this']) # 영어 단어 'this'의 임베딩 벡터값 출력

In [ ]:
print(word2vec_model['self-indulgent']) # 영어 단어 'self-indulgent'의 임베딩 벡터값 출력

## 사전 훈련된 Word2Vec을 초기 임베딩으로 사용하기

In [ ]:
import torch
import torch.nn as nn
from torchtext.vocab import Vectors

vectors = Vectors(name="eng_w2v") # 사전 훈련된 Word2Vec 모델을 vectors에 저장

TEXT.build_vocab(trainset, vectors=vectors, max_size=10000, min_freq=10) # 훈련 데이터의 단어 집합(vocabulary)를 만드는 것과 동시에 임베딩 벡터값들을 초기화

In [ ]:
print(TEXT.vocab.stoi) # 현재 단어 집합의 단어와 맵핑된 고유한 정수를 출력

In [ ]:
print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

In [ ]:
print(TEXT.vocab.vectors[0]) # <unk>의 임베딩 벡터값

In [ ]:
print(TEXT.vocab.vectors[1]) # <pad>의 임베딩 벡터값

In [ ]:
print(TEXT.vocab.vectors[10]) # this의 임베딩 벡터값(eng_w2v에 저장되어져 있던 단어 'this'의 임베딩 벡터값과 동일)

In [ ]:
print(TEXT.vocab.vectors[10000]) # 단어 'self-indulgent'의 임베딩 벡터값(env_w2v에 단어 ''self-indulgent'의 임베딩 벡터값이 존재하지 않았기 때문에 'self-indulgent'의 임베딩 벡터값은 0으로 초기화)

In [ ]:
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False) # 임베딩 벡터들이 저장되어져 있는 TEXT.vocab.vectors를 nn.Embedding()의 초기화 입력으로 사용 

In [ ]:
print(embedding_layer(torch.LongTensor([10]))) # 단어 this의 임베딩 벡터값

# 토치텍스트에서 제공하는 사전 훈련된 워드 임베딩

토치텍스트는 영어 단어들의 사전 훈련된 임베딩 벡터를 제공하고 있습니다. 다음은 제공되는 임베딩 벡터 리스트의 일부입니다.

- fasttext.en.300d
- fasttext.simple.300d
- glove.42B.300d
- glove.840B.300d
- glove.twitter.27B.25d
- glove.twitter.27B.50d
- glove.twitter.27B.100d
- glove.twitter.27B.200d
- glove.6B.50d
- glove.6B.100d
- glove.6B.200d
- glove.6B.300d

GloVe의 300차원의 임베딩 벡터들을 다운로드 받아 임베딩 벡터 초기화에 사용해봅시다.

In [ ]:
from torchtext.vocab import GloVe

TEXT.build_vocab(trainset, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(trainset)

In [ ]:
print(TEXT.vocab.stoi)

In [ ]:
print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

In [ ]:
print(TEXT.vocab.vectors[0]) # <unk>의 임베딩 벡터값

In [ ]:
print(TEXT.vocab.vectors[1]) # <pad>의 임베딩 벡터값

In [ ]:
print(TEXT.vocab.vectors[10]) # this의 임베딩 벡터값

In [ ]:
print(TEXT.vocab.vectors[10000]) # self-indulgent 임베딩 벡터값(사전 훈련된 임베딩 벡터의 단어장에 단어 'self-indulgent'이 존재하지 않았기 때문에 임베딩 벡터값은 0으로 초기화 된 상태)

In [ ]:
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)
embedding_layer(torch.LongTensor([10])) # 단어 this의 임베딩 벡터값